# Chat With Your Data

## Preserve conversation history

# Install libraries

In [ ]:
pip install openai

In [ ]:
pip install python-dotenv

In [ ]:
pip install langchain

In [ ]:
pip install langchain-openai

In [ ]:
pip install pypdf

In [ ]:
pip install faiss-cpu

In [ ]:
pip install langchainhub

## Load OpenAI API Key

In [35]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

## Prompt model with no knowledge of the Voynich manuscript

In [36]:
from langchain_openai import ChatOpenAI

#initialize the LLM we'll use - OpenAI GPT 3.5 Turbo
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")

In [37]:
#prompt the model with no additional knowledge of the Voynich manuscript beyond pretraining 
llm.invoke("What are the medicinal insights from the Voynich manuscript?")  

AIMessage(content='The Voynich manuscript is a mysterious and undeciphered text believed to have been written in the early 15th century. It contains illustrations of plants, astronomical diagrams, and what appear to be medicinal recipes. Some scholars have suggested that the manuscript may contain valuable insights into medieval medicine and herbal remedies. However, due to the inability to translate the text, the exact medicinal insights from the Voynich manuscript remain unknown. Some theories suggest that it may contain information on herbal remedies, alchemy, or other medical practices of the time period. Further research and analysis are needed to fully understand the medicinal insights that may be contained within the Voynich manuscript.', response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 19, 'total_tokens': 152}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

In [38]:
llm.invoke("What is Aetherfloris Ventus?")

AIMessage(content='Aetherfloris Ventus is a Latin term that translates to "Ether Flower Wind" in English. It is a poetic and mystical phrase that could potentially refer to a concept or element of nature that combines the qualities of ether, flowers, and wind. It may be used in literature, art, or philosophy to evoke a sense of beauty, fragility, and movement.', response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 16, 'total_tokens': 91}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

## Load vector database from disk

In [39]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


db = FAISS.load_local("../faiss_index", 
                      OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small"), 
                      allow_dangerous_deserialization=True)

## Configure retriever
### Use the similarity search capabilities of a vector store to facilitate retrieval

In [40]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

## Implement a chain
### Chain together multiple calls in a logical sequence

In [41]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [42]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#combine multiple steps in a single chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser() #convert the chat message to a string
)

## Send LLM's response to the user

In [44]:
for chunk in rag_chain.stream("What are the medicinal insights from the Voynich manuscript?"):
    print(chunk, end="", flush=True)

The Voynich manuscript contains detailed anatomical diagrams of mythical beings with annotations explaining organ functions, possibly used for medicinal or alchemical purposes. It also features illustrations of herbs with unique properties, indicating potential medicinal uses, and herbal brews and potions with specific effects on health and vitality. The manuscript offers insights into ancient medical and alchemical practices, showcasing a blend of fantastical biology and practical medicinal knowledge.

In [45]:
for chunk in rag_chain.stream("What is Aetherfloris Ventus?"):
    print(chunk, end="", flush=True)

Aetherfloris Ventus is an ethereal plant with petals lighter than air, appearing to float freely, nurtured by the winds and clouds. Its stem is nearly invisible, leading the delicate petals in a captivating ballet. The plant's essence is believed to induce lightness in the body and mind, challenging our understanding of the natural world.

In [46]:
for chunk in rag_chain.stream("What's the most important part of the Voynich manuscript?"):
    print(chunk, end="", flush=True)

The most important part of the Voynich Manuscript is the detailed anatomical diagrams of mythical beings, possibly used for medicinal or alchemical purposes, with annotations explaining the function of each organ and system. These diagrams offer a glimpse into ancient medical knowledge intertwined with fantasy, possibly serving as a practical guide for medicinal and alchemical uses. The anatomical details and annotations within the manuscript provide insights into the functions of various organs and their potential importance in ancient practices.

## Preserve Conversation History 

In [47]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """Given the chat history and a recent user question \
generate a new standalone question \
that can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed or otherwise return it as is."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

retriever_with_history = create_history_aware_retriever(
    llm, retriever, prompt
)

In [48]:
prompt

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given the chat history and a recent user question generate a new standalone question that can be understood without the chat history. Do NOT answer the question, just reformulate it if needed or otherwise return it as is.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [49]:
prompt.input_variables

['chat_history', 'input']

In [50]:
prompt.input_types

{'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}

In [51]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given the chat history and a recent user question generate a new standalone question that can be understood without the chat history. Do NOT answer the question, just reformulate it if needed or otherwise return it as is.')),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]

In [52]:
retriever_with_history

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(...), RunnableLambda(...)
| VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10c031790>, search_kwargs={'k': 6}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given the chat history and a recent user question generate a new standalone question that can be understood without the chat history. Do NOT answer the question, just reformulate it if needed or otherwise return it as is.')), MessagesPlaceholder(variable_name='cha